# Wine Spirit Project

In [ ]:
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import numpy as np
from collections import Counter

### Create Database 
1.create a database and connect to it

2.extract a set of customer supplied xlsx files

3.they saved in sql database

In [ ]:
connection = sql.connect('winespiritprojet.db')
curs = connection.cursor()

In [ ]:
ERP_df = pd.read_excel(r'E:\projet_wine_spirit\erp.xlsx') 
liaison_df = pd.read_excel(r'E:\projet_wine_spirit\liaison.xlsx')
web_df = pd.read_excel(r'E:\projet_wine_spirit\web.xlsx')

In [ ]:
ERP_df.to_sql('winespiritprojet', connection, if_exists = 'replace', index = False)
liaison_df.to_sql('winespiritprojet', connection, if_exists = 'replace', index = False)
web_df.to_sql('winespiritprojet', connection, if_exists = 'replace', index = False)

In [ ]:
connection.close()

### Analysis

we will prepare the data in order to achieve what is requested by the customer see:

-> the calculation of turnover by product and the total turnover achieved.

-> a graphic presentation of the outliers concerning the prices of the products.

-> detect any input errors.

for that we will proceed to a data cleaning, and join files

###### 1.Exploratory data analysis

###### A. le fichier Web

In [ ]:
web_df.info()

-> we see that there are several completely empty columns, an inconsistency of the data "post_content", "post_password", "post_content_filtred", "tax_class"


In [ ]:
web_df_cln = web_df.drop(columns=['post_content','post_password','post_content_filtered','tax_class'])

In [ ]:
web_df_cln.describe()

-> we see that several columns have only zero values (min, max, mean, std) "virtual","downloadable","rating_count","average_rating","post_parent","menu_order","comment_count"

In [ ]:
web_df_cln = web_df_cln.drop(columns=['virtual','downloadable','rating_count','average_rating','post_parent','menu_order','comment_count'])

In [ ]:
web_df_cln

In [ ]:
web_df_cln['post_mime_type'].value_counts()

In [ ]:
web_df_cln['ping_status'].value_counts()

In [ ]:
web_df_cln['comment_status'].value_counts()

In [ ]:
web_df_cln['post_status'].value_counts()

In [ ]:
web_df_cln['tax_status'].value_counts()

In [ ]:
web_df_cln['comment_count'].value_counts()

-> we see that some columns contain a single value and which are not relevant for our objective, so we will discard them

-> we will keep a single column for the date of the post 'post_modified_gmt'

In [ ]:
web_df_cln = web_df_cln.drop(columns = ['post_mime_type','post_modified','tax_status','post_status','comment_status','ping_status','post_date'])

In [ ]:
web_df_cln

In [ ]:
# je vérifie les doublons au niveau de la variable cible "sku"
web_df_cln.sku.duplicated().sum()

In [ ]:
# j'enlève les doublons à partir de la colonne "sku"
web_df_cln.drop_duplicates(subset = ['sku'], ignore_index = True, inplace = True)

###### B.le fichier liaison

In [ ]:
liaison_df.info()

ce fichier contient 825 lignes au totale, on constat que la colonne id_web a moins de lignes que la colonne product_id

ce fichier contient 825 lignes et le nombres de lignes par colonne et coherant 

In [ ]:
liaison_df

In [ ]:
liaison_df.isna().sum().sum() 

In [ ]:
liaison_df_cln = liaison_df.dropna().reset_index(drop = True)

In [ ]:
liaison_df_cln.product_id.duplicated().sum()

###### C.le fichier ERP

In [ ]:
ERP_df.info()

###### 3.fusion des différents fichiers

In [ ]:
fusion = web_df_cln.merge(liaison_df_cln, how='left', left_on = 'sku', right_on='id_web' )

In [ ]:
fusion

In [ ]:
fusion_all = ERP_df.merge(fusion, how='left', left_on = 'product_id', right_on='product_id' )

In [ ]:
fusion_all.sort_values('onsale_web', ascending = False, inplace = True, ignore_index = True)

In [ ]:
fusion_all

In [ ]:
fusion_na = fusion_all[fusion_all["sku"].isna()]

In [ ]:
fusion_na

In [ ]:
df_cln = fusion_all.dropna().reset_index(drop = True)

###### 4.choix de variables pertinentes.

In [ ]:
df_cln.info()

In [ ]:
df_cln = df_cln.drop(columns=['id_web','post_type','guid','post_modified_gmt','post_modified','ping_status','comment_status','post_name','post_status','post_date'])

###### 5.visualisation graphique 

In [ ]:
plt.figure(figsize=(15, 6))
sns.distplot(df_cln['price'], bins = 120, hist_kws = {'edgecolor':'black'})

In [ ]:
plt.figure(figsize = (15, 8))
sns.boxplot(x = 'price',  data = df_cln)
#sns.swarmplot(x = 'total_sales', y = 'price', data = df_cln, color = 'black')

In [ ]:
df_cln['ca_product'] = (df_cln['price'])*(df_cln['total_sales'])

In [ ]:
df_cln

In [ ]:
df_cln.price.mean()
df_cln.price.std()

In [ ]:
df_cln['zscore'] = (df_cln.price - df_cln.price.mean())/df_cln.price.std(ddof=0)

In [ ]:
df_cln

In [ ]:
outlier = df_cln[df_cln['zscore'] >= 2] 
outlier.sort_values('price', ascending = True, inplace = True, ignore_index = True)

In [ ]:
outlier

In [ ]:
q1 = df_cln.price.quantile(0.25)
q2 = df_cln.price.quantile(0.5)
q3 = df_cln.price.quantile(0.75)

iqr = q3 - q1

In [ ]:
iqr

In [ ]:
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr)

In [ ]:
upper_bound

In [ ]:
x = df_cln['price']
y = df_cln['total_sales']
z=df_cln['zscore']
z=z.astype(int)

plt.figure(figsize=(17, 6))
plt.scatter(x, y, c=z)
plt.colorbar(ticks=list(z.unique()))
plt.show()

In [ ]:
x = df_cln['price']
y = df_cln['total_sales']
z=df_cln['total_sales']
plt.figure(figsize=(17, 8))
plt.scatter(x[z ==0], y[z==0],c = 'red', alpha=0.5)
plt.scatter(x[z >0], y[z >0],c = 'green', alpha=0.5)